In [1]:
import pickle
import json
import os
from glob import glob
from tqdm import tqdm
from datasets import Audio
from collections import defaultdict

sr = 16000
audio = Audio(sampling_rate=sr)

In [2]:
pkls = sorted(glob('/home/husein/ssd2/processed-youtube-v2/*.pkl'))
len(pkls)

5145

In [3]:
with open('mapping-youtube-speakers-80.json') as fopen:
    unique_speakers = json.load(fopen)
    
len(unique_speakers)

10492

In [4]:
speakers = defaultdict(int)

for pkl in tqdm(pkls):
    with open(pkl, 'rb') as fopen:
        data = pickle.load(fopen)
        
    filename = os.path.split(pkl)[1].replace('.pkl', '')
        
    for result in data:
        speaker = result['classification_model'][1]
        y = audio.decode_example(audio.encode_example(result['wav_data']))['array']
        speaker_name = f'{filename}-{speaker}'
        if speaker_name not in unique_speakers:
            continue
        actual_speaker = unique_speakers[speaker_name]
        
        speakers[actual_speaker] += (len(y) / sr)

100%|███████████████████████████████████████| 5145/5145 [10:43<00:00,  7.99it/s]


In [5]:
speakers

defaultdict(int,
            {'Arsenal_Seakan_Tidak_Kepingin_UCL-q4uy3QER0nI-speaker 0': 261469.80287499892,
             'Ragam-Ragam_Berbuka_Puasa___Sterk_Production-kkNnTPVjhiU-speaker 9': 16.457375000000003,
             '"Berlakon_dengan_Opah_Aminah_paling_susah..._sumpah!"_-_Ayda_Jebat_a.k.a_Johana_Bukan_Gadis_Biasa-G8gUxijCp4o-speaker 0': 1.619625,
             '"Jangan_sombong_jadi_orang"_-_Mobile_Legends_\'team_player\'_Indonesia-Q5GQS6na0ug-speaker 1': 1.828625,
             'Doraemon_Malay_-_Bertukar_Mama-IISYMb-w3e0-speaker 17': 1118.8374999999999,
             '"Watch_Me_Glow!"_React_To_Dolla___SEISMIK_Challenge-ehIA9BPFq9U-speaker 2': 2.2204375,
             '"Watch_Me_Glow!"_React_To_Dolla___SEISMIK_Challenge-ehIA9BPFq9U-speaker 5': 2.2465625,
             '#AdaApaDenganAMAN-j9UvNftUQWg-speaker 2': 18.651562499999997,
             '#AdaApaDenganAMAN-j9UvNftUQWg-speaker 3': 4.258,
             'Siapa_Peneman_Para_Student_Semasa_PKP___Sterk_Production-pGjpOIeGWkY-speaker 2

In [7]:
len({k: v for k, v in speakers.items() if v > 60})

15

In [6]:
with open('lengths-80.json', 'w') as fopen:
    json.dump(speakers, fopen)